In [3]:
import torch
from pyasn1_modules.rfc2985 import pkcs_9_oc
from statsmodels.sandbox.distributions.genpareto import shape

from ultralytics import YOLO
import numpy as np
from ultralytics.nn.tasks import DetectionModel
if __name__ == "__main__":
    # 加载YOLO模型（配置文件或预训练权重）
    MODEL_PATH = r"D:\codes\YOLOS4A\ultralytics\cfg\models\S4A\yolov8-SMALL-4head-SE-CA-ECA.yaml"  # 你的配置文件路径

    model=DetectionModel(MODEL_PATH)
    # model=YOLO(MODEL_PATH)


WARNING no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  2     49920  ultralytics.nn.modules.block.C2f_SE          [64, 64, 2, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  3    284872  ultralytics.nn.modules.block.C2f_CA          [128, 128, 3, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3   1116681  ultralytics.nn.modules.block.C2f_ECA         [256, 256, 3, True]           
  7                  -1  1   1180672

In [4]:
print(model.model[0])

Conv(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)


In [5]:
print(len(model.model))

20


In [6]:
in_=torch.randn(1,3,640,640)
out=model.model[0](in_)
print(in_.shape,out.shape)

torch.Size([1, 3, 640, 640]) torch.Size([1, 32, 320, 320])


In [7]:
in_=torch.randn(1,3,640,640)
i=0
p_list=[in_]
for i in range(10):
    layer=model.model[i]
    print(f"{i}_________________________________________")
    print(f"in_shape:::{in_.shape}")
    out=layer(in_)
    print(layer)
    print(f"out_shape:::{out.shape}")
    in_=out
    p_list.append(out)
    print("##############################################")
    if i==4:
        p4_out=out
    if i==2:
        p2_out=out

0_________________________________________
in_shape:::torch.Size([1, 3, 640, 640])
Conv(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)
out_shape:::torch.Size([1, 32, 320, 320])
##############################################
1_________________________________________
in_shape:::torch.Size([1, 32, 320, 320])
Conv(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)
out_shape:::torch.Size([1, 64, 160, 160])
##############################################
2_________________________________________
in_shape:::torch.Size([1, 64, 160, 160])
C2f_SE(
  (cv1): Conv(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.0

In [8]:
p4_out.shape

torch.Size([1, 128, 80, 80])

In [9]:
p10=model.model[10]
print(p10)
print(p_list[-1].shape)
out=p10(p_list[-1])
p_list.append(out)
print(out.shape)



Upsample(scale_factor=2.0, mode='nearest')
torch.Size([1, 512, 20, 20])
torch.Size([1, 512, 40, 40])


In [10]:
p11=model.model[11]
print(p11)
print(p_list[-1].shape,print(p_list[6].shape))
out=p11([p_list[-1],p_list[6]])
p_list.append(out)
print(out.shape)

Concat()
torch.Size([1, 256, 40, 40])
torch.Size([1, 512, 40, 40]) None
torch.Size([1, 768, 40, 40])


In [11]:
p12=model.model[12]
print(p12)
print(p_list[-1].shape)
out=p12(p_list[-1])
p_list.append(out)
print(out.shape)

C2f(
  (cv1): Conv(
    (conv): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(640, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (m): ModuleList(
    (0-2): 3 x Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    )
  )

In [12]:
p13=model.model[13]
print(p13)
print("in_shape:",p_list[-1].shape)
out=p13(p_list[-1])
p_list.append(out)
print("out_shape",out.shape)

Upsample(scale_factor=2.0, mode='nearest')
in_shape: torch.Size([1, 256, 40, 40])
out_shape torch.Size([1, 256, 80, 80])


In [13]:
p14=model.model[14]
print(p14)
print("in_shape:",p_list[-1].shape,p4_out.shape)
out=p14([p_list[-1],p4_out])
p_list.append(out)
print("out_shape",out.shape)


Concat()
in_shape: torch.Size([1, 256, 80, 80]) torch.Size([1, 128, 80, 80])
out_shape torch.Size([1, 384, 80, 80])


In [14]:
p15=model.model[15]
print(p15)
print("in_shape:",p_list[-1].shape)
out=p15(p_list[-1])
p_list.append(out)
print("out_shape",out.shape)


C2f(
  (cv1): Conv(
    (conv): Conv2d(384, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(480, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (m): ModuleList(
    (0-2): 3 x Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    )
  )
)
in_

In [15]:
p16=model.model[16]
print(p16)
print("in_shape:",p_list[-1].shape)
out=p16(p_list[-1])
p_list.append(out)
print("out_shape",out.shape)


Upsample(scale_factor=2.0, mode='nearest')
in_shape: torch.Size([1, 192, 80, 80])
out_shape torch.Size([1, 192, 160, 160])


In [16]:
p17=model.model[17]
print(p17)
print("in_shape:",p_list[-1].shape,p2_out.shape)
out=p17([p_list[-1],p2_out])
p_list.append(out)
print("out_shape",out.shape)

Concat()
in_shape: torch.Size([1, 192, 160, 160]) torch.Size([1, 64, 160, 160])
out_shape torch.Size([1, 256, 160, 160])


In [17]:
p18=model.model[18]
print(p18)
print("in_shape:",p_list[-1].shape)
out=p18(p_list[-1])
p_list.append(out)
print("out_shape",out.shape)

C2f(
  (cv1): Conv(
    (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (m): ModuleList(
    (0-2): 3 x Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    )
  )
)
in_

In [18]:
for p in p_list:
    print(p.shape)

torch.Size([1, 3, 640, 640])
torch.Size([1, 32, 320, 320])
torch.Size([1, 64, 160, 160])
torch.Size([1, 64, 160, 160])
torch.Size([1, 128, 80, 80])
torch.Size([1, 128, 80, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 256, 40, 40])
torch.Size([1, 512, 20, 20])
torch.Size([1, 512, 20, 20])
torch.Size([1, 512, 20, 20])
torch.Size([1, 512, 40, 40])
torch.Size([1, 768, 40, 40])
torch.Size([1, 256, 40, 40])
torch.Size([1, 256, 80, 80])
torch.Size([1, 384, 80, 80])
torch.Size([1, 192, 80, 80])
torch.Size([1, 192, 160, 160])
torch.Size([1, 256, 160, 160])
torch.Size([1, 128, 160, 160])


In [19]:
p18_out=p_list[19]
p15_out=p_list[16]
p12_out=p_list[13]
p9_out=p_list[10]
print(p18_out.shape)
print(p15_out.shape)
print(p12_out.shape)
print(p9_out.shape)


torch.Size([1, 128, 160, 160])
torch.Size([1, 192, 80, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 512, 20, 20])


In [20]:
detect=model.model[19]
print(detect)
print("in_shape:",p18_out.shape,p15_out.shape,p12_out.shape,p9_out.shape)
out=detect([p18_out,p15_out,p12_out,p9_out])


Detect(
  (cv2): ModuleList(
    (0): Sequential(
      (0): Conv(
        (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): Sequential(
      (0): Conv(
        (conv): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     

In [21]:
len(detect.cv2)
print(detect.cv2)
##工程历史遗留问题，导致的cv不是从1开始的，1是特征提取，2是边界框回归,3是类别概率预测



ModuleList(
  (0): Sequential(
    (0): Conv(
      (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (1): Sequential(
    (0): Conv(
      (conv): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=Tru

In [22]:
print(len(detect.cv3))
detect.cv3

4


ModuleList(
  (0): Sequential(
    (0): Conv(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): Conv2d(128, 10, kernel_size=(1, 1), stride=(1, 1))
  )
  (1): Sequential(
    (0): Conv(
      (conv): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03,

In [23]:
detect.dfl

DFL(
  (conv): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
)

In [24]:
input_tensor=torch.Tensor(4,3,640,640)
output_tensor=model(input_tensor)
print(output_tensor.shape)

AttributeError: 'list' object has no attribute 'shape'

In [28]:
from PIL import Image
img=Image.open(r"C:\Users\gaoge\Desktop\小目标识别\ultralytics-main (副本)\datasets\VisDrone_yolov8n\VisDrone2019-DET-val\images\0000001_02999_d_0000005.jpg")
model=YOLO(r"C:\Users\gaoge\Desktop\小目标识别\yolov8-SMALL-4head-SE-CA-ECA_63mAP\yolov8-SMALL-4head-SE-CA-ECA_63mAP\coco\weights\best.pt")
out=model(img,conf=0.5)
out[0].show()



0: 384x640 5 cars, 1 bus, 1 bench, 71.8ms
Speed: 2.7ms preprocess, 71.8ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)


In [30]:
from PIL import Image
img=Image.open(r"C:\Users\gaoge\Desktop\小目标识别\ultralytics-main (副本)\datasets\VisDrone_yolov8n\VisDrone2019-DET-val\images\0000001_02999_d_0000005.jpg")
model=YOLO(r"D:\Models\YOLOS4A-l.pt")
out=model(img,conf=0.5)
out[0].show()


0: 384x640 1 pedestrian, 7 peoples, 10 cars, 2 trucks, 10 motors, 61.1ms
Speed: 1.8ms preprocess, 61.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)


In [32]:
detect_model=DetectionModel(MODEL_PATH)
yolo_model=YOLO(MODEL_PATH)


WARNING no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  2     49920  ultralytics.nn.modules.block.C2f_SE          [64, 64, 2, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  3    284872  ultralytics.nn.modules.block.C2f_CA          [128, 128, 3, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3   1116681  ultralytics.nn.modules.block.C2f_ECA         [256, 256, 3, True]           
  7                  -1  1   1180672

In [33]:
yolo_model.model==detect_model.model

False